# Calculate velocity distribution and position destribution of MDCK spheroids

Data  
2D time-series images of flowing spheroids  
- Hydrodynamic focusing and acoustic focusing: on
- Time points: 0, 5 min  
- Number of measurements at each time point: 3
- Use only frames from 0 to 360 (=nine seconds)

## Libraries

In [ ]:
import glob
import os
import re
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread, imshow
from skimage.filters import median, gaussian

import trackpy as tp

## Functions

In [ ]:
def make_dataset(time, measurement, start_frame=0, end_frame=360):
    '''
    Make a list of data to analyze

    time: timepoint of the measurement to analyze
    measurement: id of the measurement to analyze
    start_frame: first frame of the data to analyze
    end_frame: last frame of the data to analyze
    '''
    imgs = [os.path.abspath(p) for p in glob.glob(f'{path_data}id557_{time}min_{measurement}*.tif')]
    imgs = sorted(imgs, key = lambda x : re.search(r'_00[0-9][0-9][0-9].', x).group())
    imgs = imgs[start_frame:end_frame]
    imgs = [imread (img) for img in imgs]
    
    return imgs

def clip(img, bounds=(None, None)):
    '''
    Clip image along the first axis so that the image includes only the inside of the microfluidic channel

    img: image to process
    bounds: coordinates on which image is clipped
    '''
    return img[bounds[0]:bounds[1], :]

def filter_img(img):
    '''
    Apply median and Gaussian filter to image
    '''
    filtered_img = median(img)
    filtered_img = gaussian(filtered_img, sigma=1)
    return filtered_img

def process(imgs, bounds=(None, None)):
    '''
    Processs images before analysis

    imgs: images to analyze
    bounds: coordinates on which images are clipped
    '''
    filtered_imgs = [filter_img(clip(np.fliplr(img), bounds=bounds)) for img in imgs]
    return filtered_imgs

def test(time, measurement, end_frame=20, bounds=(None, None), threshold=20/255):
    '''
    Analyze the small subset of the data and chack the result

    time: timepoint of the measurement to analyze
    measurement: id of the measurement to analyze
    end_frame: last frame of the data to analyze
    bounds: coordinates on which image is clipped
    threshold: threshold value to detect flowing objects
    '''
    imgs = make_dataset(time, measurement, end_frame=end_frame)
    imgs = process(imgs, bounds)
    locations = tp.batch(imgs, diameter=25, threshold=threshold)
    pred = tp.predict.ChannelPredict(110, 'x', minsamples=10)
    track = pred.link_df(locations, search_range=[13, 40], memory=10)
    track = tp.filter_stubs(track, 5)
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 10))
    tp.annotate(locations[locations['frame']==0], imgs[0], ax=axes[0])
    tp.plot_traj(track)

def analysis(time, measurement, bounds=(None, None), threshold=20/255):
    '''
    Analyze the first 360 frames of the measurement

    time: timepoint of the measurement to analyze
    measurement: id of the measurement to analyze
    bounds: coordinates on which image is clipped
    threshold: threshold value to detect flowing objects
    '''
    imgs = make_dataset(time, measurement)
    imgs = process(imgs, bounds)
    locations = tp.batch(imgs, diameter=25, threshold=threshold)
    pred = tp.predict.ChannelPredict(110, 'x', minsamples=10)
    track = pred.link_df(locations, search_range=[13, 40], memory=10)
    track = tp.filter_stubs(track, 120)
    imsd = tp.motion.imsd(track, mpp=1/1.124, fps=40, max_lagtime=360)
    print('The number of detected tracks is:', len(imsd.columns))
        
    return track, imsd

## Directories

In [ ]:
path_data = '../../data/2D/long/'
path_result = '../../result/'

## Test using the limited number of frames

In [ ]:
img = imread(path_data+'id557_0min_1*_00001.tif')
plt.figure(figsize=(20, 10))
plt.imshow(img[235:235+1124, :])

In [ ]:
wall_up = 235 # coordinate of the upper wall
wall_bottom = wall_up + 1124 # coordinate of the bottom wall

In [ ]:
test(0, measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
test(5, measurement=1, bounds=(wall_up, wall_bottom))

## Analysis

### 0 min

In [ ]:
wall_up = 235
wall_bottom = wall_up + 1124

time = 0

In [ ]:
measurement = 1

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)

In [ ]:
measurement = 2

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)

In [ ]:
measurement = 3

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)

### 5min

In [ ]:
wall_up = 235
wall_bottom = wall_up + 1124

time = 5

In [ ]:
measurement = 1

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)

In [ ]:
measurement = 2

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)

In [ ]:
measurement = 3

In [ ]:
track, imsd = analysis(time=time, measurement=measurement, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(track)

In [ ]:
imsd.to_csv(f'{path_result}id557_{time}min_{measurement}_imsd.csv')
with open(f'{path_result}id557_{time}min_{measurement}_track.txt', 'wb') as f:
    pickle.dump(track, f)